# Úkol č. 2 - předzpracování dat a binární klasifikace (do 9. listopadu 23:59)

  * V rámci tohoto úkolu se musíte vypořádat s příznaky, které jsou různých typů.
  * Před tím, než na nich postavíte predikční model, je třeba je nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na podmnožiny vhodné k trénování modelu.
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.
  * Ukázka prvních řádků souboru *results.csv*:
  
```
ID,survived
1000,0
1001,1
...
```

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí křížové validace. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor s predikcemi pro vyhodnocovací data (`results.csv`).
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import KNNImputer

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier


import sklearn.metrics as metrics
from sklearn.model_selection import ParameterGrid

random_seed = 727

* data si zde nactu, provedu jen deterministicke, jednoduche doplnovani chybejicich hodnot
* slozitejsi doplneni Age probiha pote co se rozdeli na trenovaci a validacni
* cely dataset si rozdelim na 90% + 10%
   * 10% rezervuju na testovani
   * 90% je pool dat pro rozdeleni pro cross validaci

funkce pro predpracovani dat
* nejake sloupce jsem odstranil uplne (name, ticket, home dest) ty jsem povazoval za nepodstatne v predikci preziti
* cabin jsem nahrail 1 pokud osoba ma kajutu, 0 pokud nema
* fare z typu double na int jednoduchym vynasobenim
* dle googlu se nejdrive zastavilo v Southampton, pote Cherbourg a nakonec Queenstown, proto takove poradi, ostatni (NaN jsou -1)

In [2]:
def preprocessSimple(data):
    data = data.drop(columns = ['ticket', 'name', 'home.dest'])
    #-----------------------------------------
    data['sex'] = data['sex'].replace({'male': 0, 'female':1})
    #-----------------------------------------
    data['cabin'] = data['cabin'].fillna(-1).apply(lambda x: 1 if x != -1 else 0) #has or doesnt have a cabin
    #-----------------------------------------
    data['fare'] = data['fare'].fillna(0).apply(lambda x: x*10000).astype('int64')
    #-----------------------------------------
    data['embarked'] = data['embarked'].fillna(0).replace({'S' : 1, 'C':2, 'Q' : 3})
    
    return data

In [3]:
data = preprocessSimple(pd.read_csv("data.csv").drop(columns = ['ID']))

#split 90+10
X_rest, X_test, y_rest, y_test = train_test_split(
    data.drop(columns=['survived']), data['survived'], test_size=0.1, random_state=random_seed
)

trainData = X_rest.assign(Survived= y_rest)

k predikci preziti jsem pouzil 4 metody, kazda metoda jeste je spojena s hyperparametry

In [4]:
classifiers = [(DecisionTreeClassifier, {'max_depth': range(1,101), 'criterion': ['entropy', 'gini']}),
               (RandomForestClassifier, {'n_estimators': range(1, 100, 5), 'max_depth': range(1, 5)}),
               (AdaBoostClassifier,  {'n_estimators': range(1,100,5), 'learning_rate': [0.01, 0.05, 0.1, 0.3, 0.5, 1]}),
               (KNeighborsClassifier, {'n_neighbors' : range(2, 100)})]
n_splits = 5

* testovani vsech modelu a jejich ruzne kombinace hyperparametru
* validacni hodnota se bere jako prumer validacnich presnosti pomoci krizove validace
* rozdeleni X_rest (90% dat) pomoci KFold
* jako vysledek metody s urcitym hyperparametrem beru prumer
* nalezeni vhodneho hyperparametru pomoci maxarg

In [5]:
#returns filled and scaled train dataset
#with it also returns a function that should be used to fill and scale other datasets
#1st scales and then fills missing values
def transformDataFunctionCreation(method, train):
    if method.__name__ != 'KNeighborsClassifier':
        def scale(x):
            return x
    else:
        scaler = MinMaxScaler()
        train = pd.DataFrame(scaler.fit_transform(train),index=train.index, columns=train.columns)
        def scale(x):
            return pd.DataFrame(scaler.transform(x),index=x.index, columns=x.columns)

    imputer = KNNImputer(n_neighbors=5, weights='distance')
    train = pd.DataFrame(imputer.fit_transform(train),index=train.index, columns=train.columns)
    
    for col in train.select_dtypes('float64').columns:
        train[col] = train[col].astype('int64')
        
    def transformFunction(x):
        x = scale(x)
        x = pd.DataFrame( imputer.transform(x),index=x.index, columns=x.columns)
        for col in x.select_dtypes('float64').columns:
            x[col] = x[col].astype('int64')
        return x
        
    
    return train, transformFunction

In [6]:
#returns mean validation of accuracy while testing 1 concrete method with 1 set of params
#uses KFold for cross validation
def GetValAcc(method, param, data):
    val_p = []
    for train, val in KFold(n_splits=n_splits).split(data.index):
        
        train_x = data[data.index.isin(train)].drop(columns=['Survived'])
        train_y = data[data.index.isin(train)]['Survived']

        val_x = data[data.index.isin(val)].drop(columns=['Survived'])
        val_y = data[data.index.isin(val)]['Survived']
        
        #scales and fills missing values on train_x
        #also gets a function that can be applied on validation set to fill its missing value
        train_x, FillAndTransform = transformDataFunctionCreation(method, train_x)

        dt = method(**param).fit(train_x, train_y)

        #fills missing values(and scales) before predicting
        val_x = FillAndTransform(val_x)
        val_p.append(metrics.accuracy_score(dt.predict(val_x), val_y))
        
    return np.mean(val_p)


def TestClassifier(method, p, data):
    param_comb = ParameterGrid(p)
    
    #checks all values with different params using same method and calculates the best param for best value
    val_acc = [GetValAcc(method, param, data) for param in param_comb]
    best = np.argmax(val_acc)
    
    return param_comb[best], val_acc[best]

In [7]:
#train + test
#train split using KFold for cross validation

bestMethod = ('', '', -1)

for method, p in classifiers:
    best_params, bestVal = TestClassifier(method, p, trainData)
        
    print('used', method.__name__)
    print(best_params)
    print('mean validation accuracy =', '{0:.6f}'.format(bestVal))
    
    print('----------------------------')
    
    if bestVal > bestMethod[2]:
        bestMethod = (method, best_params, bestVal)
        
print('best was', bestMethod[0].__name__, 'with', bestMethod[1])

used DecisionTreeClassifier
{'max_depth': 3, 'criterion': 'entropy'}
mean validation accuracy = 0.802589
----------------------------
used RandomForestClassifier
{'n_estimators': 96, 'max_depth': 4}
mean validation accuracy = 0.801514
----------------------------
used AdaBoostClassifier
{'n_estimators': 11, 'learning_rate': 0.5}
mean validation accuracy = 0.792545
----------------------------
used KNeighborsClassifier
{'n_neighbors': 33}
mean validation accuracy = 0.789088
----------------------------
best was DecisionTreeClassifier with {'max_depth': 3, 'criterion': 'entropy'}


* zde uz si jen kontroluju vysledek nad test data
* vyuzil jsem celych zbylych 90% dat k nauceni modelu a ten pak pouzil k predikci

In [8]:
method = bestMethod[0]
params = bestMethod[1]

train_x, TransFunc = transformDataFunctionCreation(method, X_rest)
test_x = TransFunc(X_test)

dt = method(**params).fit(train_x, y_rest)
print('accuracy on test data:', metrics.accuracy_score(dt.predict(test_x), y_test) )


accuracy on test data: 0.85


In [9]:
evaluation = pd.read_csv("evaluation.csv")
IDList = evaluation['ID']

evaluation = preprocessSimple(evaluation.drop(columns = ['ID']))
evaluation = TransFunc(evaluation)

res = dt.predict(evaluation)

resDF = pd.DataFrame(list(zip(IDList, res)), 
               columns =['ID', 'survived'])
resDF.to_csv(r'result.csv', index=False)